## Serving frontend: make a frontend that loads the “model”, takes request,
computes its embeddings, and returns image file name + description.

In [1]:
import os, ast
import numpy as np
import pandas as pd
import ollama
from PIL import Image
from IPython.display import display

def load_model_csv(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    def parse_embedding(v):
        # stored as string like "[0.1, 0.2, ...]"
        return np.array(ast.literal_eval(v), dtype=np.float32)

    df["embedding"] = df["embedding"].apply(parse_embedding)
    return df

def compute_embedding(text: str, model: str = "embeddinggemma") -> np.ndarray:
    r = ollama.embeddings(model=model, prompt=text)
    emb = r["embedding"] if isinstance(r, dict) else r.embedding
    return np.array(emb, dtype=np.float32)

def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    denom = (np.linalg.norm(a) * np.linalg.norm(b))
    if denom == 0:
        return -1.0
    return float(np.dot(a, b) / denom)

def find_most_similar(df_model: pd.DataFrame, query_emb: np.ndarray):
    best_score = -1e9
    best_row = None

    for _, row in df_model.iterrows():
        score = cosine_similarity(row["embedding"], query_emb)
        if score > best_score:
            best_score = score
            best_row = row

    return best_row["filename"], best_row["description"], best_score


In [4]:
def serve_query(query: str, model_csv: str = "image_model.csv", image_dir: str = "frames"):
    # load model
    df_model = load_model_csv(model_csv)

    # embed query
    q_emb = compute_embedding(query)

    # retrieve best match
    filename, description, score = find_most_similar(df_model, q_emb)

    # show results
    print("Query:", query)
    print("Best match:", filename, "score:", score)
    print("Best description:", description)

    # show image
    img_path = os.path.join(image_dir, filename)
    if os.path.exists(img_path):
        display(Image.open(img_path))
    else:
        print("WARNING: Image file not found at:", img_path)

    return filename, description, score

In [6]:
serve_query("Show me a bear", model_csv="image_model.csv", image_dir="frames")

Query: Show me a bear
Best match: frames/frame_0009.jpg score: 0.5132958292961121
Best description: This image depicts a brown bear resting or lounging on a rocky outcrop. The bear appears to be in a natural, forested environment, possibly during a transitional season like autumn, given the mix of green and autumnal foliage in the background. The bear is positioned with its back to the camera, looking slightly to the side.


('frames/frame_0009.jpg',
 'This image depicts a brown bear resting or lounging on a rocky outcrop. The bear appears to be in a natural, forested environment, possibly during a transitional season like autumn, given the mix of green and autumnal foliage in the background. The bear is positioned with its back to the camera, looking slightly to the side.',
 0.5132958292961121)